## Data Science Immesive Final Project (General Assembly ATX Spring 2017)

#### Idea. Predict time on trail from features of dataset  
## What sort of things can I add to this dataset? Weather,etc..
My data was obtained through the Appalachian Trail Conservancy. I called and asked for all availible data on Thru hikers. Talk about form. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
at = pd.read_excel('BenHikersInfo.xlsx')

In [ ]:
at.head()

#### Exploratory Analysis

In [ ]:
at['Yr Comp'].isnull().sum()

In [ ]:
at.describe()

In [ ]:
# Null values by column 
plt.figure(figsize = (10,10))
z = at.isnull().sum()
z.plot(kind = 'bar')

In [ ]:
at['Hiking_Disability'].nunique()

# How I have cleaned this DataSet: 
I explore the data column by column to understand what sort of missing or inaccurate values are contained.

def (clean_1): Columns that can/will be dropped
- Rename columns to be more accurate.
- Drop 'Days Elapsed' column as it does not accuratly calculate the number of days on trail. (I replace this column in clean_2 with 'Total_Days')
- Drop 'Hiking_Disability' Column as it was not well reported ()
- Drop Dog Weight Column (16238 of possible 17835 values were 0)
- Drop columns with too many null values (columns that exceed 85 percent null values)
- drop null values out of columns
- Drop Under2kAmount column since (16125 out of 17835) values were 0 essentially the same as null values 


def (clean_2/clean_3): Clean up the Age column 
- Fill null values in 'Age' column with median age.
- All values below 4(There were some negative values) 
- The youngest person to thru-hike the AT is 5 so I assume all values above this are 

 
    
def (clean_4): Datetime objects, 
-  Convert 'Start' and 'Finish' columns to Datetime objects
-  Create a new column 'Total Days' that calculates total days on trail (at['Finish] - at['Start'])) 
-  There was as single null value in completion year, however, the finish date showed the hiker completed in 2016 so I changed the value to 2016 
- I conveted the completion yr column from floats to ints 



def (clean_5): Clean up 'Sex' column
- In (clean_1) I dropped all null values in this column 
- Drop 'oher' and 'othe' out of the sex column
- Convert the 'Sex' column to numerical format with the map function










To do 
1.) # Find out whether the section column included is a thru section. 
2.) Dropped a column that had a finish date of 2022 as year 17369
3.) Figure out how to combine lime, noro, and giardia 




## Data Exploration

## Cleaning Dataset

In [ ]:
def clean_1(df):
    df.drop(labels = df[['Disease Other','School','DogWeightLbs','Days Elapsed','MinorityComments','School',
                         'DogBreed','Disability','Under2KAmount']], axis = 1, inplace = True)
    df.rename(columns = {'AgeYrFin': 'Age','Race or Heritage': 'Race',
                        'StAbbrev':'State', 'Hiking Disability ': 'Hiking_Disability', 
                         'Hike Type': 'Hike_Type', 'Yr Comp':'completion_yr',
                         'Hiking Disability':'Hiking_Disability'},inplace = True)
    df.dropna(subset= ['Start','Sex','Race','Country','State','Hike_Type'],inplace = True)

clean_1(at)  

In [ ]:
def clean_2(df):
    df['Age'] = df['Age'].fillna(df['Age'].median())
clean_2(at)


def clean_3(x,median_age): 
    if x < 4:
        return median_age
    else:
        return x 
at['Age'] = at['Age'].apply(lambda x: clean_3(x, at['Age'].median()))

In [ ]:
def clean_4(df):
    df['Finish'] = pd.to_datetime(df['Finish'])
    df['Start'] = pd.to_datetime(df['Start'])
    df['Total_Days'] = df['Finish'] - df['Start']
    df['completion_yr'].fillna(2016,inplace = True)
    df['completion_yr'] = df['completion_yr'].astype(int)
clean_4(at)

In [ ]:
def clean_5(df):
    df['Sex'] = df['Sex'].map({'M':0,'F':1})
    to_drop = ['othe','Oher']
    x = df[df['Sex'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_5(at)

In [ ]:
at['Sex'].value_counts()

In [ ]:
# drop hike types that are 'Alternative','alternative','Alterntive'
def clean_5(df):
    to_drop = ['Alternative','alternative','Alterntive']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_5(at)



In [ ]:

df.drop(df.index[[17264,17236,17369]], inplace = True)

In [ ]:
at.ix[17107].to_frame().T


In [ ]:
df.drop([17264,17236])

In [ ]:
#plt.figure(figsize = (10,10))
#null.plot(kind = 'bar')

# Notes on How I have cleaned my DataSet






4.) Drop Dog Weight Column 16238 of possible 17835 values were 0. So nearly the same as null 

5.) Drop Under2kAmount column 16125 of 17835 values were 0. It was just not self reported very well

4.) Rename Columns with spaces in them or w/ confusing names. 

5.) Replace Null Values in yr completed with 2016 since we do know his start date* 

6.) Drop oher and othe out of the sex column

7.) Convert null values in age column to mean. I checked out a distplot for how this would change
with using mean,median or mode, and not alot of change. 

8.) Dropped a column that had a finish date of 2022 as year 17369

9.)

10.) # Find out whether the section column included is a thru section. 

11.)

12.)

13.)

In [ ]:
2.576 * .0087

In [ ]:
def clean_6(df):
    if df < 4:
        return df['Age'].fillna(df['Age'].mean())
at.apply(clean_6)

